**Name:** Albright Maduka

**Student ID:** 9053136  

**Course:** CSCN 8020  (Reinforcement Learning Programming)

**Solution to Assignment 1**

In [11]:
# Importing libraries
import time
import random
import logging
import numpy as np
import os
os.makedirs("logs", exist_ok=True)

## Problem 1: Pick-and-Place Robot as an MDP

We model the task as a Markov Decision Process (MDP):
(MDP) = (S, A, P, R, γ), where γ = 0.95 is the discount factor.

### State (S)
To satisfy the Markov property and allow smooth control, the state includes:
s_t = (θ_t, θ̇_t, x^ee_t, ẋ^ee_t, g_t, x^obj_t, x^target_t)

Where:
- θ_t: joint angles
- θ̇_t: joint velocities (needed for smooth motion)
- x^ee_t, ẋ^ee_t: end-effector pose and velocity
- g_t: gripper state (holding object or not)
- x^obj_t: object pose
- x^target_t: target pose

This state is Markov because it contains all information needed to predict future states.

### Action  (A)
The agent controls the robot motors directly. A natural choice is:
a_t = τ_t ∈ R^n
where τ_t are joint torques (one per joint).
Alternatives include motor voltages or desired joint velocities.

### Transition Model P(s' | s, a)
The transition is given by robot dynamics:
s_{t+1} = f(s_t, a_t) + ε
where ε represents noise, friction, and modeling errors.
Collisions, joint limits, or contacts may clamp the next state or terminate the episode.

### Reward Function R(s, a, s')
We want fast, smooth, safe, and successful execution:
R_t =
+100 if the object is placed correctly
+20  if a successful grasp is achieved
-λ1 ||τ_t||^2            (penalize large torques)
-λ2 ||τ_t - τ_{t-1}||^2  (penalize jerky motions)
-λ3 Δt                   (penalize time to encourage speed)
-200 if collision or object drop occurs

This reward encourages:
- Correct task completion
- Smooth and energy-efficient motion
- Safety (avoid collisions)
- Speed (finish quickly)

### Termination
An episode terminates when:
- The object is placed successfully, or
- A collision, or
- A time limit is reached.


## Problem 2: 2×2 Gridworld

I have four states arranged as:

s1  s2
s3  s4

Rewards:
R(s1) = 5, R(s2) = 10, R(s3) = 1, R(s4) = 2
Discount factor: γ = 0.95

Transitions are deterministic. Invalid moves keep the agent in the same state.

Although Problem 2 refers to Value Iteration, it also mentions policy evaluation and policy improvement. Therefore, both approaches are presented here for completeness.

---

### A) Two Sweeps of Value Iteration

Bellman optimality update:
V_{k+1}(s) = R(s) + γ * max_a V_k(s')

Initialize:
V0(s) = 0 for all s

#### Sweep 1
Since V0(s') = 0:
V1(s) = R(s)

So:
V1(s1) = 5
V1(s2) = 10
V1(s3) = 1
V1(s4) = 2

#### Sweep 2
Compute best next-state values:

- From s1: best next is s2 with value 10
  V2(s1) = 5 + 0.95 * 10 = 14.5

- From s2: best next is staying in s2 (invalid moves keep you there)
  V2(s2) = 10 + 0.95 * 10 = 19.5

- From s3: best next is s1 with value 5
  V2(s3) = 1 + 0.95 * 5 = 5.75

- From s4: best next is s2 with value 10
  V2(s4) = 2 + 0.95 * 10 = 11.5

So after two sweeps:
V2(s1) = 14.5
V2(s2) = 19.5
V2(s3) = 5.75
V2(s4) = 11.5

---

### B) Policy Evaluation + Policy Improvement

Initial policy π0: always move UP in every state.

#### Round 1 — Policy Evaluation

Transitions under π0:
s1 → s1, s2 → s2, s3 → s1, s4 → s2

Solve:
V(s1) = 5 + 0.95 V(s1)  → V(s1) = 100
V(s2) = 10 + 0.95 V(s2) → V(s2) = 200
V(s3) = 1 + 0.95 V(s1)  → V(s3) = 96
V(s4) = 2 + 0.95 V(s2)  → V(s4) = 192

#### Round 1 — Policy Improvement

Choose best action in each state:
s1 → Right (to s2)
s2 → Up (stay)
s3 → Right (to s4)
s4 → Up (to s2)

So:
π1 = { s1:R, s2:U, s3:R, s4:U }

#### Round 2 — Policy Evaluation

V(s2) = 200
V(s1) = 5 + 0.95 * 200 = 195
V(s4) = 2 + 0.95 * 200 = 192
V(s3) = 1 + 0.95 * 192 = 183.4

#### Round 2 — Policy Improvement

At s3:
Up → s1 gives 195
Right → s4 gives 192
So choose Up.

Final improved policy:
π2 = { s1:R, s2:U, s3:U, s4:U }


### Problem 3: 5×5 Gridworld with Value Iteration

In this problem, we consider a 5×5 gridworld Markov Decision Process (MDP) with deterministic transitions. States are identified by their (row, column) indices. The goal (terminal) state is at (4,4) with reward +10. Three grey (non-favourable) states, (2,2), (3,0), and (0,4), have reward −5, while all other states have reward −1. If an action would move the agent outside the grid, the agent remains in the same state.

The discount factor is set to \( gamma = 0.95 \). We first define the reward function based on whether a state is terminal, grey, or a regular state. Then, we apply Value Iteration to compute the optimal state-value function \( V^* \) and the corresponding optimal policy \( pi^* \). The stopping criterion is based on a small tolerance \( theta \), and convergence is detected when the maximum change in values across all states becomes smaller than \( \theta \).

In addition to the standard (synchronous) Value Iteration, we also implement an in-place version, where updated values are immediately reused within the same sweep. Both methods are run until convergence, and the resulting \( V^* \) and \( pi^* \) are printed as 5×5 tables.

Finally, we compare the two methods in terms of the number of iterations and computation time. In this experiment, both synchronous and in-place Value Iteration converge to the same optimal value function and policy, and they require the same number of iterations due to the small size of the grid. The computational complexity per sweep is \( O(|S||A|) \), where \( |S| \) is the number of states and \( |A| \) is the number of actions.

In [ ]:
# Problem 3: 5x5 Gridworld
# ENVIRONMENT SPECIFICATION

# Grid size 5x5 (states identified by (row, col))
N = 5

# Terminal/goal state (episode ends when reached)
GOAL = (4, 4)

# Grey (non-favourable) states (valid states but reward = -5)
GREY = {(2, 2), (3, 0), (0, 4)}

# Action set: Right, Down, Left, Up
# We represent each action as a (dr, dc) movement on the grid.
ACTIONS = {
    "R": (0, 1),
    "D": (1, 0),
    "L": (0, -1),
    "U": (-1, 0),
}

def in_bounds(r: int, c: int) -> bool:
    """Return True if (r,c) is inside the grid."""
    return 0 <= r < N and 0 <= c < N

def reward(s):
    """
    Reward function R(s) as described in the assignment:
    +10 if s is GOAL
    -5  if s is in GREY
    -1  otherwise
    """
    if s == GOAL:
        return 10
    if s in GREY:
        return -5
    return -1

def step(s, a):
    """
    Deterministic transition:
    - If action is valid -> move to next state
    - If action hits a wall -> stay in same state
    - If already at terminal -> remain in terminal (absorbing)
    """
    if s == GOAL:
        return s  # absorbing terminal
    dr, dc = ACTIONS[a]
    r, c = s
    nr, nc = r + dr, c + dc
    if not in_bounds(nr, nc):
        return s  # invalid move -> stay
    return (nr, nc)

def all_states():
    """List of all states in the grid."""
    return [(r, c) for r in range(N) for c in range(N)]

# UTILITY: FORMATTING TABLES


def V_to_grid(V):
    """
    Convert V dict {(r,c): value} into a numpy array for printing/logging.
    """
    grid = np.zeros((N, N), dtype=float)
    for r in range(N):
        for c in range(N):
            grid[r, c] = V[(r, c)]
    return grid

def greedy_policy_from_V(V):
    """
    Derive greedy policy pi(s) = argmax_a V(s') using deterministic transitions.
    If tie: Python's max() picks first by internal order; that's fine unless your rubric demands tie-handling.
    """
    pi = {}
    for s in all_states():
        if s == GOAL:
            pi[s] = "T"  # terminal
        else:
            pi[s] = max(ACTIONS.keys(), key=lambda a: V[step(s, a)])
    return pi

def policy_to_grid(pi):
    """Convert policy dict into 5x5 grid of action symbols for printing/logging."""
    grid = np.empty((N, N), dtype=object)
    for r in range(N):
        for c in range(N):
            grid[r, c] = pi[(r, c)]
    return grid


# VALUE ITERATION (SYNCHRONOUS)

def value_iteration_sync(gamma=0.95, theta=1e-8, max_iters=100000, logger=None):
    """
    Synchronous (two-array) Value Iteration:
    - compute V_new from V (old) for all states
    - then replace V <- V_new at the end of sweep

    Stopping rule:
    - stop if delta = max_s |V_new(s) - V(s)| < theta
    """
    S = all_states()
    V = {s: 0.0 for s in S}  # initialize values to 0

    t0 = time.time()
    for it in range(1, max_iters + 1):
        delta = 0.0
        V_new = V.copy()

        for s in S:
            if s == GOAL:
                # Terminal state value: set it to +10 (consistent with reward and terminal semantics)
                V_new[s] = 10.0
                continue

            # Bellman optimality backup:
            # V(s) = R(s) + gamma * max_a V(s')
            best_next = max(V[step(s, a)] for a in ACTIONS)
            V_new[s] = reward(s) + gamma * best_next

            delta = max(delta, abs(V_new[s] - V[s]))

        V = V_new

        # Log every iteration (you can change to log every k iterations if you want smaller logs)
        if logger:
            logger.info(f"[VI-SYNC] iter={it:05d} delta={delta:.12f}")

        if delta < theta:
            elapsed = time.time() - t0
            return V, it, elapsed

    elapsed = time.time() - t0
    return V, max_iters, elapsed


# VALUE ITERATION (IN-PLACE)


def value_iteration_inplace(gamma=0.95, theta=1e-8, max_iters=100000, logger=None):
    """
    In-place Value Iteration:
    - update V(s) immediately and use updated values in the same sweep

    Often converges in fewer sweeps due to faster propagation of new information.
    """
    S = all_states()
    V = {s: 0.0 for s in S}

    t0 = time.time()
    for it in range(1, max_iters + 1):
        delta = 0.0

        for s in S:
            old = V[s]

            if s == GOAL:
                V[s] = 10.0
            else:
                best_next = max(V[step(s, a)] for a in ACTIONS)
                V[s] = reward(s) + gamma * best_next

            delta = max(delta, abs(V[s] - old))

        if logger:
            logger.info(f"[VI-INPLACE] iter={it:05d} delta={delta:.12f}")

        if delta < theta:
            elapsed = time.time() - t0
            return V, it, elapsed

    elapsed = time.time() - t0
    return V, max_iters, elapsed


#  MAIN: RUN + LOG + REPORT

def main():
    # Logging to logs/problem3.log
    logging.basicConfig(
        filename="logs/problem3.log",
        filemode="w",
        level=logging.INFO,
        format="%(asctime)s | %(message)s",
        force=True  # IMPORTANT in notebooks
    )
    logger = logging.getLogger("Problem3")

    # Parameters (explicit for marking)
    gamma = 0.95
    theta = 1e-8
    max_iters = 100000

    logger.info("==========  Problem 3 ==========")
    logger.info(f"Grid size: {N}x{N}")
    logger.info(f"Goal state: {GOAL} (terminal)")
    logger.info(f"Grey states: {sorted(GREY)}")
    logger.info("Rewards: +10 at GOAL, -5 at GREY, -1 otherwise")
    logger.info(f"Parameters: gamma={gamma}, theta={theta}, max_iters={max_iters}")
    logger.info("Actions: R, D, L, U (deterministic); invalid move => stay")
    logger.info("------------------------------------------------------")

    # 1) Synchronous VI
    V_sync, it_sync, t_sync = value_iteration_sync(
        gamma=gamma, theta=theta, max_iters=max_iters, logger=logger
    )
    pi_sync = greedy_policy_from_V(V_sync)

    logger.info("----- Final Results: Synchronous Value Iteration -----")
    logger.info(f"Converged in {it_sync} iterations, time={t_sync:.6f} sec")
    logger.info("V* (rounded to 6):\n" + str(np.round(V_to_grid(V_sync), 6)))
    logger.info("pi*:\n" + str(policy_to_grid(pi_sync)))

    # 2) In-place VI
    V_inp, it_inp, t_inp = value_iteration_inplace(
        gamma=gamma, theta=theta, max_iters=max_iters, logger=logger
    )
    pi_inp = greedy_policy_from_V(V_inp)

    logger.info("----- Final Results: In-Place Value Iteration -----")
    logger.info(f"Converged in {it_inp} iterations, time={t_inp:.6f} sec")
    logger.info("V* (rounded to 6):\n" + str(np.round(V_to_grid(V_inp), 6)))
    logger.info("pi*:\n" + str(policy_to_grid(pi_inp)))

    # 3) Comparison
    max_abs_diff = max(abs(V_sync[s] - V_inp[s]) for s in V_sync)
    same_policy = all(pi_sync[s] == pi_inp[s] for s in pi_sync)

    logger.info("----- Comparison (Sync vs In-Place) -----")
    logger.info(f"Max abs difference in V: {max_abs_diff:.12e}")
    logger.info(f"Policies identical? {same_policy}")
    logger.info(f"Sync: iterations={it_sync}, time={t_sync:.6f}")
    logger.info(f"InPlace: iterations={it_inp}, time={t_inp:.6f}")
    logger.info("======================================================")

    # Console output
    print("Problem 3 complete. Log written to: logs/problem3.log")
    print("\nV* (γ=0.95):\n", np.round(V_to_grid(V_sync), 6))
    print("\nπ*:\n", policy_to_grid(pi_sync))
    print("\nComparison:")
    print("  Max |V_sync - V_inplace| =", f"{max_abs_diff:.3e}")
    print("  Policies identical?      =", same_policy)
    print("  Sync iterations/time     =", it_sync, f"{t_sync:.6f}s")
    print("  InPlace iterations/time  =", it_inp, f"{t_inp:.6f}s")

main()


Problem 3 complete. Log written to: logs/problem3.log

V* (γ=0.95):
 [[-0.097387  0.950119  2.052757  3.213428  0.435187]
 [ 0.950119  2.052757  3.213428  4.435187  5.72125 ]
 [ 2.052757  3.213428  0.435187  5.72125   7.075   ]
 [-0.786572  4.435187  5.72125   7.075     8.5     ]
 [ 4.435187  5.72125   7.075     8.5      10.      ]]

π*:
 [['R' 'R' 'R' 'D' 'D']
 ['R' 'R' 'R' 'R' 'D']
 ['R' 'D' 'R' 'R' 'D']
 ['R' 'R' 'R' 'R' 'D']
 ['R' 'R' 'R' 'R' 'T']]

Comparison:
  Max |V_sync - V_inplace| = 0.000e+00
  Policies identical?      = True
  Sync iterations/time     = 10 0.005190s
  InPlace iterations/time  = 10 0.002002s


### Results and Discussion

The optimal policy mainly selects **Right (R)** and **Down (D)** actions, which move the agent efficiently toward the goal at (4,4). The goal state has value 10, which is consistent with the terminal reward, while the grey states have noticeably lower values (including a negative value near (3,0)) due to the −5 penalty, making those regions less desirable.

Both the synchronous and in-place implementations of Value Iteration converged to the same optimal value function and policy under the tolerance \( \theta = 10^{-8} \). In this experiment, both methods required the same number of iterations, which can occur in small deterministic problems. Since each sweep evaluates all states and actions, the computational cost per sweep is \( O(|S||A|) \), meaning it grows proportionally with the number of states and actions.

### Problem 4: Off-policy Monte Carlo with Importance Sampling

This section:
1. Defines the same 5×5 Gridworld as in Problem 3.
2. Implements off-policy Monte Carlo with Weighted Importance Sampling.
3. Estimates the state-value function V(s) from the learned action-value function Q(s,a).
4. Compares the Monte Carlo estimate with the optimal value function V* obtained from Value Iteration.
5. Logs the results to `problem4.log`.

In [ ]:
# Problem 4: Off-policy Monte Carlo with Importance Sampling (Weighted IS)
# ENVIRONMENT SPECIFICATION

# Grid size: 5x5
N = 5

# Terminal/goal state (episode ends when reached)
GOAL = (4, 4)

# Grey (non-favourable) states with penalty -5
GREY = {(2, 2), (3, 0), (0, 4)}

# Action set: Right, Down, Left, Up
# Each action is represented as (dr, dc) movement on the grid
ACTIONS = {
    "R": (0, 1),
    "D": (1, 0),
    "L": (0, -1),
    "U": (-1, 0),
}

def in_bounds(r, c):
    """Check if (r,c) is inside the grid."""
    return 0 <= r < N and 0 <= c < N

def reward(s):
    """
    Reward function R(s) (state-based, as defined in the assignment):
    +10 for goal state,
    -5 for grey states,
    -1 for all other states.
    """
    if s == GOAL:
        return 10
    if s in GREY:
        return -5
    return -1

def step(s, a):
    """
    Deterministic transition function.
    If action would leave the grid, the agent stays in the same state.
    If state is terminal, it remains there.
    """
    if s == GOAL:
        return s
    dr, dc = ACTIONS[a]
    r, c = s
    nr, nc = r + dr, c + dc
    if not in_bounds(nr, nc):
        return s
    return (nr, nc)

def all_states():
    """Return a list of all states in the grid."""
    return [(r, c) for r in range(N) for c in range(N)]

def V_to_grid(V):
    """Convert dictionary V(s) to a 5x5 numpy grid for printing."""
    grid = np.zeros((N, N), dtype=float)
    for r in range(N):
        for c in range(N):
            grid[r, c] = V[(r, c)]
    return grid

def policy_to_grid(pi):
    """Convert policy dictionary to a 5x5 grid of actions."""
    return np.array([[pi[(r, c)] for c in range(N)] for r in range(N)], dtype=object)

# VALUE ITERATION (FOR COMPARISON AGAINST MC)

def value_iteration_sync(gamma=0.95, theta=1e-8, max_iters=100000):
    """
    Standard synchronous Value Iteration to compute the optimal V*.
    Used for comparison against the Monte Carlo estimate.

    gamma: discount factor
    theta: convergence tolerance (stop when max change < theta)
    """
    S = all_states()
    V = {s: 0.0 for s in S}

    for _ in range(max_iters):
        delta = 0.0
        V_new = V.copy()

        for s in S:
            if s == GOAL:
                V_new[s] = 10.0
                continue

            best_next = max(V[step(s, a)] for a in ACTIONS)
            V_new[s] = reward(s) + gamma * best_next
            delta = max(delta, abs(V_new[s] - V[s]))

        V = V_new
        if delta < theta:
            break

    return V

# BEHAVIOR POLICY b(a|s)

def behavior_policy(_s):
    """
    Fixed behavior policy b(a|s): uniform random over {R, D, L, U}.
    """
    return random.choice(list(ACTIONS.keys()))

def behavior_prob(_s, _a):
    """Probability under behavior policy (uniform over 4 actions)."""
    return 1.0 / len(ACTIONS)

# OFF-POLICY MC (WEIGHTED IMPORTANCE SAMPLING)

def off_policy_mc_weighted_is(
    gamma=0.95,
    num_episodes=50000,
    max_steps=200,
    seed=0,
    log_every=1000,
    logger=None,
):
    """
    Off-policy Monte Carlo control with Weighted Importance Sampling (WIS).

    Q(s,a): action-value function
    C(s,a): cumulative IS weights

    Target policy pi: greedy w.r.t Q (deterministic).
    Behavior policy b: uniform random.

    Backward update:
      G = gamma*G + r
      C(s,a) += W
      Q(s,a) += (W/C(s,a)) * (G - Q(s,a))

    Weight update (when action matches greedy):
      W *= (pi/b) = 1 / 0.25 = 4
    """
    random.seed(seed)
    np.random.seed(seed)

    S = all_states()
    Q = {(s, a): 0.0 for s in S for a in ACTIONS}
    C = {(s, a): 0.0 for s in S for a in ACTIONS}

    def greedy_action(s):
        """Greedy action under current Q."""
        return max(ACTIONS.keys(), key=lambda a: Q[(s, a)])

    t0 = time.time()
    recent_lengths = []
    recent_maxW = 0.0

    for ep in range(1, num_episodes + 1):

        # Generate one episode using behavior policy b
        s = (0, 0)   # start state
        episode = [] # list of (s, a, r)

        for _ in range(max_steps):
            a = behavior_policy(s)

            # NOTE: rewards are state-based R(s) in this assignment
            r = reward(s)
            episode.append((s, a, r))

            if s == GOAL:
                break
            s = step(s, a)

        recent_lengths.append(len(episode))

        # Backward return updates
        G = 0.0
        W = 1.0

        for (s, a, r) in reversed(episode):
            G = gamma * G + r

            C[(s, a)] += W
            Q[(s, a)] += (W / C[(s, a)]) * (G - Q[(s, a)])

            # If not greedy action under target policy, pi(a|s)=0 -> stop
            if a != greedy_action(s):
                break

            # W = W * (pi/b). Here pi=1 and b=1/4
            W *= (1.0 / behavior_prob(s, a))
            recent_maxW = max(recent_maxW, W)

        # Logging progress
        if logger and ep % log_every == 0:
            avg_len = float(np.mean(recent_lengths)) if recent_lengths else 0.0
            logger.info(
                f"[MC-WIS] ep={ep:06d} avg_len={avg_len:.2f} recent_maxW={recent_maxW:.2f}"
            )
            recent_lengths = []
            recent_maxW = 0.0

    elapsed = time.time() - t0

    # Derive V(s) and greedy policy pi(s) from Q
    V = {s: max(Q[(s, a)] for a in ACTIONS) for s in S}
    pi = {s: ("T" if s == GOAL else greedy_action(s)) for s in S}

    return V, pi, elapsed

# MAIN: RUN + LOG + REPORT

def main():
    # Logging to logs/problem4.log
    logging.basicConfig(
        filename="logs/problem4.log",
        filemode="w",
        level=logging.INFO,
        format="%(asctime)s | %(message)s",
        force=True  # IMPORTANT in notebooks
    )
    logger = logging.getLogger("Problem4")

    gamma = 0.95
    episodes = 50000
    max_steps = 200
    seed = 0
    log_every = 1000

    logger.info("========== Problem 4 ==========")
    logger.info(f"Grid size: {N}x{N}")
    logger.info(f"Goal state: {GOAL} (terminal)")
    logger.info(f"Grey states: {sorted(GREY)}")
    logger.info("Rewards: +10 at GOAL, -5 at GREY, -1 otherwise")
    logger.info("Behavior policy b(a|s): uniform random over {R,D,L,U} -> b=0.25 each")
    logger.info("Target policy pi(a|s): greedy wrt Q (deterministic)")
    logger.info("Importance Sampling: Weighted IS (lower variance than ordinary IS)")
    logger.info(f"Parameters: gamma={gamma}, episodes={episodes}, max_steps={max_steps}, seed={seed}")
    logger.info("------------------------------------------------------")

    # Run Monte Carlo (WIS)
    V_mc, pi_mc, t_mc = off_policy_mc_weighted_is(
        gamma=gamma,
        num_episodes=episodes,
        max_steps=max_steps,
        seed=seed,
        log_every=log_every,
        logger=logger
    )

    logger.info("----- Final Results (MC Estimate) -----")
    logger.info(f"Time={t_mc:.6f} sec")
    logger.info("V_MC (rounded to 3):\n" + str(np.round(V_to_grid(V_mc), 3)))
    logger.info("pi_MC:\n" + str(policy_to_grid(pi_mc)))

    # Compare MC estimate to Value Iteration optimum
    t0_vi = time.time()
    V_vi = value_iteration_sync(gamma=gamma, theta=1e-8)
    t_vi = time.time() - t0_vi

    grid_mc = V_to_grid(V_mc)
    grid_vi = V_to_grid(V_vi)

    mae = float(np.mean(np.abs(grid_mc - grid_vi)))
    max_err = float(np.max(np.abs(grid_mc - grid_vi)))

    logger.info("----- Comparison: MC vs Value Iteration -----")
    logger.info(f"VI time={t_vi:.6f} sec")
    logger.info(f"MAE(|V_MC - V*|) = {mae:.6f}")
    logger.info(f"Max(|V_MC - V*|) = {max_err:.6f}")
    logger.info("======================================================")

    # Console output
    print("Problem 4 complete. Log written to: logs/problem4.log")
    print("\nV_MC (γ=0.95):\n", np.round(grid_mc, 3))
    print("\nπ_MC:\n", policy_to_grid(pi_mc))
    print("\nComparison vs VI (same γ):")
    print("  VI time =", f"{t_vi:.6f}s")
    print("  MAE(|V_MC - V*|) =", f"{mae:.6f}")
    print("  Max(|V_MC - V*|) =", f"{max_err:.6f}")

main() 


Problem 4 complete. Log written to: logs/problem4.log

V_MC (γ=0.95):
 [[-0.102  0.938  2.025  3.172  0.403]
 [ 0.896  2.012  3.181  4.401  5.693]
 [ 1.857  3.155  0.414  5.696  7.05 ]
 [-0.816  4.394  5.69   7.052  8.486]
 [ 4.166  5.673  7.046  8.49  10.   ]]

π_MC:
 [['D' 'R' 'D' 'D' 'D']
 ['D' 'R' 'R' 'D' 'D']
 ['R' 'D' 'R' 'D' 'D']
 ['R' 'R' 'R' 'D' 'D']
 ['R' 'R' 'R' 'R' 'T']]

Comparison vs VI (same γ):
  VI time = 0.000000s
  MAE(|V_MC - V*|) = 0.045186
  Max(|V_MC - V*|) = 0.269031


### Results and Discussion

Using off-policy Monte Carlo with Weighted Importance Sampling and a discount factor \( gamma = 0.95 \), the estimated value function \( V_{\{MC}} \) and greedy policy \( pi_{\{MC}} \) were obtained, with the goal state value close to 10 and the grey states showing lower values due to the −5 penalty. The learned policy mainly selects **Right (R)** and **Down (D)** actions, similar to the optimal policy from Value Iteration. Comparing \( V_{\{MC}} \) with the optimal value function \( V^* \) gives a mean absolute error of about 0.045 and a maximum error of about 0.269, indicating that Monte Carlo provides a close but approximate solution due to sampling noise and a finite number of episodes. In terms of performance, Monte Carlo methods require many episodes and their cost grows with episode length, whereas Value Iteration is a model-based method with per-iteration complexity \( O(|S||A|) \) that converges in a small number of sweeps for this problem, highlighting the trade-off between model-free sampling methods and dynamic programming approaches.